## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-22-53-16 +0000,bbc,Putin hosts Trump negotiators at Kremlin for '...,https://www.bbc.com/news/articles/cwyv0ex7v8ro...
1,2025-12-02-22-52-41 +0000,nypost,Gov. Gavin Newsom trolls Trump with phony new ...,https://nypost.com/2025/12/02/us-news/gov-gavi...
2,2025-12-02-22-51-15 +0000,nyt,Hegseth Says He Did Not See Survivors of Boat ...,https://www.nytimes.com/2025/12/02/us/politics...
3,2025-12-02-22-51-00 +0000,wsj,Opinion | Trump’s Pardon for Cocaine Juan,https://www.wsj.com/opinion/juan-orlando-herna...
4,2025-12-02-22-50-00 +0000,wsj,Opinion | Trump Strikes a Good Drug Deal,https://www.wsj.com/opinion/donald-trump-u-k-d...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,67
13,new,23
17,he,16
102,up,16
22,strike,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
332,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,135
303,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,134
14,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...,130
320,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...,128
19,2025-12-02-22-27-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,124


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
332,135,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...
125,87,2025-12-02-19-29-00 +0000,wsj,Defense Secretary Pete Hegseth said he didn’t ...,https://www.wsj.com/politics/national-security...
306,61,2025-12-02-03-11-29 +0000,nyt,Kristi Noem Recommends New Travel Ban After Na...,https://www.nytimes.com/2025/12/01/us/politics...
76,57,2025-12-02-20-55-50 +0000,nypost,Luigi Mangione appears to nervously bite nails...,https://nypost.com/2025/12/02/us-news/luigi-ma...
147,52,2025-12-02-18-23-52 +0000,cbc,"Ex-Honduran president, convicted of traffickin...",https://www.cbc.ca/news/world/us-honduras-hern...
14,47,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...
303,45,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
177,43,2025-12-02-16-04-52 +0000,nypost,Samsung unveils first triple-folding phone ahe...,https://nypost.com/2025/12/02/business/samsung...
100,41,2025-12-02-20-08-00 +0000,wsj,Oil Futures Fall Amid Russia-Ukraine Peace Talks,https://www.wsj.com/finance/commodities-future...
64,38,2025-12-02-21-27-21 +0000,nypost,Nicolas Maduro could end up in gas-rich Qatar ...,https://nypost.com/2025/12/02/us-news/qatar-fl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
